In [1]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "colab"


def plot3D_4x(P, TITLES):

    fig = make_subplots(
        rows=1, cols=4,
        specs=[[{'type': 'surface'},{'type': 'surface'},{'type': 'surface'},{'type': 'surface'}]],
        subplot_titles=TITLES)# , subplot_titles=(titles))
    
    omega = 1
    q = 41
    
    fig.add_trace(
    go.Surface(z=P[0], colorscale='viridis', showscale=False),
    row=1, col=1)
    
    fig.add_trace(
    go.Surface(z=P[1], colorscale='viridis', showscale=False),
    row=1, col=2)
    
    fig.add_trace(
    go.Surface(z=P[2], colorscale='viridis', showscale=False),
    row=1, col=3)
    
    fig.add_trace(
    go.Surface(z=P[3], colorscale='viridis', showscale=False),
    row=1, col=4)
    
    fig.update_scenes(xaxis_title_text='ek',  
                      yaxis_title_text='vk')
        
    camera = dict(
        eye=dict(x=2, y=1.5, z=1.)
    )
    aspectratio = dict(x=1, y=1., z=1.5)
    
        
    fig.layout.scene1.camera = camera
    fig.layout.scene2.camera = camera
    fig.layout.scene3.camera = camera
    fig.layout.scene4.camera = camera
    fig.layout.scene1.aspectratio=aspectratio
    fig.layout.scene2.aspectratio=aspectratio
    fig.layout.scene3.aspectratio=aspectratio
    fig.layout.scene4.aspectratio=aspectratio
    
    fig.update_layout(
        scene1 = dict(zaxis = dict(nticks=4),),
        scene2 = dict(zaxis = dict(nticks=4),),
        scene3 = dict(zaxis = dict(nticks=4),),
        scene4 = dict(zaxis = dict(nticks=4),),
        title=r'',
        autosize=True,
        width=1800, height=600,
        margin=dict(l=10, r=5, b=0, t=35))
    
    fig.show()



In [2]:
from scipy.optimize import curve_fit

def fit_ImSigma_2(iw, gamma, alpha, beta):
    ImSigma = -gamma - alpha * iw - beta * iw**2
    return ImSigma

def fit_ImSigma_1(iw, gamma, alpha):
    ImSigma = gamma - alpha * iw
    return ImSigma

In [3]:
from torch.utils.data import Dataset
import torch
import numpy as np
import h5py
import pytorch_lightning as pl
from torch import nn
import matplotlib.pyplot as plt
%matplotlib inline


import sys, re
sys.path.append('/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/')

import torch
import json
from torch.utils.data import DataLoader
import os

def create_datasets(config):
    
    PATH = config["PATH_TRAIN"]

    # f = h5py.File(PATH, 'r')
    # data = np.array(f["train"]['data'][:1000])
    # print("************************************")
    # print("Size of dataset: ", data.shape)
    # print("************************************")
    # train, validation = torch.utils.data.random_split(data, [int(data.__len__()*0.8), int(data.__len__())-int(data.__len__()*0.8)], generator=torch.Generator().manual_seed(42))

    PATH = config["PATH_TRAIN"]
    f = h5py.File(PATH, 'r')
    if config["TRAINDATA"]==config["VALIDATIONDATA"]:
        data = np.array(f[config["TRAINDATA"]])
        train, validation = torch.utils.data.random_split(data, [int(data.__len__()*config["SPLIT"]), int(data.__len__())-int(data.__len__()*config["SPLIT"])], generator=torch.Generator().manual_seed(42))
    else:
        train = np.array(f[config["TRAINDATA"]])
        validation = np.array(f[config["VALIDATIONDATA"]])
    
    return train, validation

def load_model_data(SAVEPATH):
    ep = 0
    for filename in os.listdir(SAVEPATH+"checkpoints"):
        epc = int(re.findall(r'\d+', filename)[0])
        # if epc == 0:
        #     f = os.path.join(SAVEPATH, filename)
        #     FILENAME = "checkpoints/"+filename
        if epc > ep:
            f = os.path.join(SAVEPATH, filename)
            FILENAME = "checkpoints/"+filename
            ep = epc
    config = json.load(open(SAVEPATH+'config.json'))
    print(SAVEPATH+FILENAME)
    print(config["DATA_LOADER"])
    checkpoint = torch.load(SAVEPATH+FILENAME, map_location=torch.device('cpu'))
    print(" Checkpoint loaded...")
    
    ''' Model setup '''
    wrapers = __import__("src.wrappers.wrapers", fromlist=['object'])#.wrapers
    print(" working 1 ...")
    model = getattr(wrapers, config["MODEL_WRAPER"])(config)
    print(" Model loading... ")
    model.load_state_dict(checkpoint['state_dict'])
    print(" Model loaded... ")
    # p = kk
    # model = 0

    ''' Dataloading '''
    train_data, validation_data = create_datasets(config)
    train_data = np.array(train_data)
    validation_data = np.array(validation_data)
    
    ### > Single HDF5 file containing training and validation data 
    ld = __import__("load_data", fromlist=['object'])
    # data_set = load_data.Dataset_ae(config)
    train_set = getattr(ld, config["DATA_LOADER"])(config, train_data)
    validation_set = getattr(ld, config["DATA_LOADER"])(config, validation_data)

    train_dataloader = DataLoader(train_set, batch_size=22, shuffle=False)
    validation_dataloader = DataLoader(validation_set, batch_size=22, shuffle=False)
    
    # f = h5py.File(config["PATH_TRAIN"], 'r')
    # parameters = f["valid"]["parameters"]
    return model, train_set, validation_set, train_dataloader, validation_dataloader, config

SAVEPATH = "/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/saves/GTrans/metiso/U2c0_b30_N250000/save_GNN_basis_2_BS20_2024-08-16/version_0/"
model, train_set, validation_set, train_dataloader, validation_dataloader, config = load_model_data(SAVEPATH)
model.model.eval()


print("...loading done.")

/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/saves/GTrans/metiso/U2c0_b30_N250000/save_GNN_basis_2_BS20_2024-08-16/version_0/checkpoints/epoch=95-step=473280.ckpt
Dataset_graph_generic
 Checkpoint loaded...
 working 1 ...
 Model loading... 
 Model loaded... 
...loading done.


In [4]:

for n,d in enumerate(train_set):
    print(d["node_feature"][0][600:605])
    if n == 5: break

# print(train_set[0])
# for n,d in enumerate(train_dataloader):
#     print(d)
#     print(d[0][:5])
#     if n == 5: break
    

tensor([-1.6224, -0.6014, -0.3686, -0.2688, -0.2144])
tensor([-0.4353, -0.1466, -0.0897, -0.0659, -0.0531])
tensor([-1.5534, -0.5657, -0.3495, -0.2585, -0.2098])
tensor([-0.9864, -1.0119, -0.7367, -0.5675, -0.4635])
tensor([-0.6272, -0.2180, -0.1411, -0.1117, -0.0979])
tensor([-0.0114, -0.0064, -0.0069, -0.0082, -0.0097])


In [5]:
PATH = "/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/data/SigmaTrans/U2c0_b30_N250000.hdf5"
f = h5py.File(PATH, 'r')

train = np.array(f["metiso/data/30.0"])
valid = np.array(f["transprox/data/30.0"])

train_para = np.array(f["metiso/parameters/30.0"])
valid_para = np.array(f["transprox/parameters/30.0"])

# print(valid_para[1])

### Unique parameters
unique_ek = []
unique_vk = [] 
unique_beta = [] 

for n,para in enumerate(valid_para):
    if para[1] not in unique_ek:
        unique_ek.append(para[1])
    if para[2] not in unique_vk:
        unique_vk.append(para[2])
    if para[0] not in unique_beta:
        unique_beta.append(para[0])

for n,para in enumerate(train_para):
    if para[1] not in unique_ek:
        unique_ek.append(para[1])
    if para[2] not in unique_vk:
        unique_vk.append(para[2])
    if para[0] not in unique_beta:
        unique_beta.append(para[0])

unique_beta = np.array(unique_beta)
unique_ek = np.array(unique_ek)
unique_vk = np.array(unique_vk)



In [6]:
data = {}

for n,para in enumerate(valid_para):
    if para[1] not in data.keys():
        data[para[1]] = {}
    if para[2] not in data[para[1]].keys():
        data[para[1]][para[2]] = {}

    data[para[1]][para[2]]["input"] = torch.tensor(valid[n][0])
    data[para[1]][para[2]]["target"] = torch.tensor(valid[n][1])
    data[para[1]][para[2]]["param"] = [para[1], para[2]]
    data[para[1]][para[2]]["set"] = 1

for n,para in enumerate(train_para):
    if para[1] not in data.keys():
        data[para[1]] = {}
    if para[2] not in data[para[1]].keys():
        data[para[1]][para[2]] = {}

    data[para[1]][para[2]]["input"] = torch.tensor(train[n][0])
    data[para[1]][para[2]]["target"] = torch.tensor(train[n][1])
    data[para[1]][para[2]]["param"] = [para[1], para[2]]
    data[para[1]][para[2]]["set"] = -1

sort_ek = np.sort(unique_ek)
sort_vk = np.sort(unique_vk)

data_np = np.zeros((sort_ek.shape[0],sort_vk.shape[0],3,1000), dtype=complex)
para_np = np.zeros((sort_ek.shape[0],sort_vk.shape[0],2))
trainvalid = np.zeros((sort_ek.shape[0],sort_vk.shape[0]))


for n in range(0,sort_ek.shape[0]):
    if n%100==0: print(n)
    for m in range(0,sort_vk.shape[0]):
        trainvalid[n,m] = data[sort_ek[n]][sort_vk[m]]["set"]



0
100
200
300
400


In [7]:
for n,d in enumerate(train_set):
    print(d["node_feature"][0][600:605])
    if n == 5: break

print(data[sort_ek[0]][sort_vk[0]]["target"][:5].imag)


tensor([-1.6224, -0.6014, -0.3686, -0.2688, -0.2144])
tensor([-0.4353, -0.1466, -0.0897, -0.0659, -0.0531])
tensor([-1.5534, -0.5657, -0.3495, -0.2585, -0.2098])
tensor([-0.9864, -1.0119, -0.7367, -0.5675, -0.4635])
tensor([-0.6272, -0.2180, -0.1411, -0.1117, -0.0979])
tensor([-0.0114, -0.0064, -0.0069, -0.0082, -0.0097])
tensor([-9.5493, -3.1831, -1.9099, -1.3642, -1.0610], dtype=torch.float64)


In [8]:
for n in range(0,sort_ek.shape[0]):
    if n%100==0: print(n)
    for m in range(0,sort_vk.shape[0]):
        data_np[n,m,0,:] = data[sort_ek[n]][sort_vk[m]]["input"]
        data_np[n,m,1,:] = data[sort_ek[n]][sort_vk[m]]["target"]
        # data_np[n,m,2,:] = model(data[sort_ek[n]][sort_vk[m]]["input"].imag.float()).detach().numpy()
        para_np[n,m,0] = sort_ek[n]
        para_np[n,m,1] = sort_vk[m]

0


/tmp/ipykernel_1259188/2543809242.py:4: DeprecationWarning:

__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.

/tmp/ipykernel_1259188/2543809242.py:5: DeprecationWarning:

__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.



100
200
300
400


In [12]:
# # data_torch = torch.zeros((500,500,300))
# ld = __import__("load_data", fromlist=['object'])

# for n in range(0,sort_ek.shape[0]):
    
#     if n%100==0: print(n)
#     sample1 = data_np[0,:,:2,:]
#     sample1 = getattr(ld, config["DATA_LOADER"])(config, sample1)
#     # print(sample1[n]["node_feature"].shape)
#     # print(sample1[n]["edge_index"].shape)
#     # print(sample1[n]["vectors"].shape)
#     # print(sample1[n]["target"].shape)
#     graph_sample = {}
#     graph_sample["node_feature"] = sample1[n]["node_feature"]#[None]
#     graph_sample["edge_index"] = sample1[n]["edge_index"]#[None]
#     graph_sample["vectors"] = sample1[n]["vectors"]#[None]
#     graph_sample["target"] = sample1[n]["target"]#[None]

#     # for m in range(0,sort_vk.shape[0]):
#     #     if m%50==0: print(n,m)
#     #     graph_sample["node_feature"] = sample1[m]["node_feature"]#[None]
#     #     data_torch[n,m,:] = model(graph_sample)#.detach().numpy()

0


KeyboardInterrupt: 

In [13]:
for n in range(0,sort_ek.shape[0]):
    
    if n%100==0: print(n)
    sample1 = data_np[0,:,:2,:]
    sample1 = getattr(ld, config["DATA_LOADER"])(config, sample1)
    graph_sample = {}
    graph_sample["node_feature"] = torch.zeros((500,300, 900))
    graph_sample["edge_index"] = torch.zeros((500,2, 90000), dtype=int)
    graph_sample["vectors"] = torch.zeros((500,300, 300))
    graph_sample["target"] = torch.zeros((500,300))
    for m in range(0,sort_vk.shape[0]):
        if m%50==0: print(n,m)
        graph_sample["node_feature"][m] = sample1[m]["node_feature"].float()#[None]
        graph_sample["edge_index"][m] = sample1[m]["edge_index"].float()#[None]
        graph_sample["vectors"][m] = sample1[m]["vectors"].float()#[None]
        graph_sample["target"][m] = sample1[m]["target"].float()#[None]
    # graph_sample["node_feature"] = torch.tensor(graph_sample["node_feature"])
    # graph_sample["edge_index"] = torch.tensor(graph_sample["edge_index"])
    # graph_sample["vectors"] = torch.tensor(graph_sample["vectors"])
    # graph_sample["target"] = torch.tensor(graph_sample["target"])
    print(model(graph_sample).shape)
    break
    # data_torch[n,:,:] = model(graph_sample)#.detach().numpy()

0
0 0
0 50
0 100
0 150
0 200
0 250
0 300
0 350
0 400
0 450


ValueError: Expected 'edge_index' to be two-dimensional (got 3 dimensions)

In [ ]:
ivmax = 300
loss = torch.nn.MSELoss()
mse_loss = torch.zeros((sort_ek.shape[0], sort_vk.shape[0]))
for n in range(0,sort_ek.shape[0]):
    if n%100==0: print(n)
    for m in range(0,sort_vk.shape[0]):
        mse_loss[n,m] = loss(torch.tensor(data_np[n,m,1,:ivmax].imag), torch.tensor(data_np[n,m,2,:ivmax].imag))

In [ ]:
n = np.linspace(0,999,1000)
iv = (2*n+1)/30*np.pi
mx = 5

fpara_pred = torch.zeros((sort_ek.shape[0], sort_vk.shape[0], 2))
fpara_target = torch.zeros((sort_ek.shape[0], sort_vk.shape[0], 2))
Z_loss = torch.zeros((sort_ek.shape[0], sort_vk.shape[0]))
gamma_loss = torch.zeros((sort_ek.shape[0], sort_vk.shape[0]))


for n in range(0,sort_ek.shape[0]):
    if n%100==0: print(n)
    for m in range(0,sort_vk.shape[0]):
        ta2, _ = curve_fit(fit_ImSigma_2, iv[:mx], data_np[n,m,1,:mx].imag)
        pe2, _ = curve_fit(fit_ImSigma_2, iv[:mx], data_np[n,m,2,:mx])
        fpara_target[n,m,0] = ta2[0]
        fpara_target[n,m,1] = ta2[1]
        fpara_pred[n,m,0] = pe2[0]
        fpara_pred[n,m,1] = pe2[1]

        gamma_loss[n,m] = loss(torch.tensor(ta2[0]), torch.tensor(pe2[0]))
        Z_loss[n,m] = loss(torch.tensor(ta2[1]), torch.tensor(pe2[1]))

In [ ]:
for n in range(0,sort_ek.shape[0]):
    if n%100==0: print(n)
    for m in range(0,sort_vk.shape[0]):
        gamma_loss[n,m] = np.sqrt( abs(fpara_target[n,m,0]**2 - fpara_pred[n,m,0]**2) )
        Z_loss[n,m] = np.sqrt( abs(fpara_target[n,m,1]**2 - fpara_pred[n,m,1]**2) )


In [ ]:
TITLES = ["MSE", "Gamma Loss", "Z Loss", "Train/Valid"]
plot3D_4x([mse_loss, gamma_loss, Z_loss, trainvalid],TITLES)
TITLES = ["Zt", "Zp", "Gt", "Gp"]
plot3D_4x([fpara_target[:,:,0], fpara_pred[:,:,0], fpara_target[:,:,1], fpara_pred[:,:,1]], TITLES)


In [ ]:
beta = 30
n = np.linspace(0,999,1000)
iv = (2*n+1)/30*np.pi
mx = 5


fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(18,4))
fign = 0
ax[fign].plot(iv[:mx], data_np[450,310,1,:mx].imag)
ax[fign].plot(iv[:mx], data_np[450,310,2,:mx], "--")

fign = 1
mx = 1000
ax[fign].plot(iv[:mx], data_np[450,310,1,:mx].imag)
ax[fign].plot(iv[:mx], data_np[450,310,2,:mx], "--")

In [ ]:
# for n in range(0,sort_ek.shape[0]):
#     if n%100==0: print(n)
#     for m in range(0,sort_vk.shape[0]):
#         print(Z_loss[n,m], fpara_target[n,m,0], fpara_pred[n,m,0])
# max_index = np.unravel_index(np.argmax(matrix), matrix.shape)
# print(np.unravel_index(np.argmax(Z_loss[:,:]), Z_loss.shape))


print(Z_loss[249,30], fpara_target[249,30,1], fpara_pred[249,30,1], loss(fpara_target[249,30,1], fpara_pred[249,30,1]))


In [ ]:
random_sample = int(np.random.rand()*len(validation_set))
print(random_sample)
graph_sample = {}
graph_sample["node_feature"] = validation_set[random_sample]["node_feature"][None]
graph_sample["edge_index"] = validation_set[random_sample]["edge_index"][None]
graph_sample["vectors"] = validation_set[random_sample]["vectors"][None]
graph_sample["target"] = validation_set[random_sample]["target"][None]
pred_ae_1 = model.model(graph_sample)
emb_vec = model.model.vec_embedding_mlp(graph_sample["vectors"][0]).detach().numpy()

fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(18,4))
# print(graph_sample["target"].shape)
# print(pred_ae_1.detach().numpy().shape)
# print(graph_sample["target"].shape)

fign = 0
ax[fign].plot(graph_sample["target"][0,:])
ax[fign].plot(pred_ae_1.detach().numpy()[:300])

# print(graph_sample["vectors"].shape)

fign = 1
ax[fign].plot(graph_sample["node_feature"][0,12,:])
# ax[fign].plot(graph_sample["vectors"][0,2])
# ax[fign].plot(graph_sample["vectors"][0,12])
# ax[fign].plot(graph_sample["vector"][20])

fign = 2
ax[fign].plot(emb_vec[110,:])
ax[fign].plot(emb_vec[221,:])
ax[fign].plot(emb_vec[0,:])
